# LookML Multi-Entity Demo (DuckDB)

This notebook builds a small DuckDB database, defines a multi-entity LookML model (orders, customers, products), loads it with Sidemantic, and produces a simple chart.


Prereqs (Colab):

- `pip install sidemantic duckdb polars pyarrow altair`


In [ ]:
# Colab setup
!pip -q install sidemantic duckdb polars pyarrow altair

In [ ]:
import tempfile
from pathlib import Path

import duckdb

workdir = Path(tempfile.mkdtemp(prefix="sidemantic_lookml_demo_"))
db_path = workdir / "demo.duckdb"

con = duckdb.connect(str(db_path))
con.execute(
    """
    create table customers as
    select * from (values
        (1, 'Ava', 'US'),
        (2, 'Noah', 'US'),
        (3, 'Liam', 'CA'),
        (4, 'Mia', 'GB')
    ) as t(id, name, region);

    create table products as
    select * from (values
        (10, 'Widget', 'hardware'),
        (11, 'Gizmo', 'hardware'),
        (12, 'Cloud', 'software')
    ) as t(id, name, category);

    create table orders as
    select * from (values
        (100, '2025-01-01', 'completed', 120.0, 1, 10),
        (101, '2025-01-02', 'completed', 80.0, 2, 11),
        (102, '2025-01-03', 'returned', 60.0, 3, 12),
        (103, '2025-01-04', 'completed', 200.0, 1, 12),
        (104, '2025-01-05', 'pending', 50.0, 4, 10)
    ) as t(id, order_date, status, amount, customer_id, product_id);
    """
)

orders_df = con.execute("select * from orders").pl()
con.close()

orders_df

In [ ]:
lookml_dir = workdir / "lookml"
lookml_dir.mkdir(parents=True, exist_ok=True)

# Compact LookML syntax (no ${TABLE} references) to keep SQL clean for DuckDB
views = """
view: orders {
  sql_table_name: orders ;;

  dimension: id {
    primary_key: yes
    type: number
    sql: id ;;
  }

  dimension_group: order_date {
    type: time
    timeframes: [date, month]
    sql: order_date ;;
  }

  dimension: status {
    type: string
    sql: status ;;
  }

  dimension: customer_id {
    type: number
    sql: customer_id ;;
  }

  dimension: product_id {
    type: number
    sql: product_id ;;
  }

  measure: order_count {
    type: count
  }

  measure: revenue {
    type: sum
    sql: amount ;;
  }
}

view: customers {
  sql_table_name: customers ;;

  dimension: id {
    primary_key: yes
    type: number
    sql: id ;;
  }

  dimension: name {
    type: string
    sql: name ;;
  }

  dimension: region {
    type: string
    sql: region ;;
  }
}

view: products {
  sql_table_name: products ;;

  dimension: id {
    primary_key: yes
    type: number
    sql: id ;;
  }

  dimension: name {
    type: string
    sql: name ;;
  }

  dimension: category {
    type: string
    sql: category ;;
  }
}
""".strip()

explores = """
explore: orders {
  join: customers {
    sql_on: ${orders.customer_id} = ${customers.id} ;;
    relationship: many_to_one
    type: left_outer
  }

  join: products {
    sql_on: ${orders.product_id} = ${products.id} ;;
    relationship: many_to_one
    type: left_outer
  }
}
""".strip()

(lookml_dir / "views.lkml").write_text(views)
(lookml_dir / "explores.lkml").write_text(explores)

lookml_dir

In [ ]:
from sidemantic import SemanticLayer
from sidemantic.adapters.lookml import LookMLAdapter

adapter = LookMLAdapter()
graph = adapter.parse(lookml_dir)

# Explore joins are parsed after views, so rebuild adjacency for joins
graph.build_adjacency()

layer = SemanticLayer(connection=f"duckdb:///{db_path}")
layer.graph = graph

layer.list_models()

In [ ]:
result = layer.query(
    metrics=["orders.revenue", "orders.order_count"],
    dimensions=["customers.region", "products.category"],
)

df = result.pl()
df

In [ ]:
import altair as alt
import polars as pl

# Altair needs JSON-serializable types (cast Decimal -> Float)
df_plot = df.with_columns(
    pl.col("revenue").cast(pl.Float64),
    pl.col("order_count").cast(pl.Float64),
)

alt.Chart(alt.Data(values=df_plot.to_dicts())).mark_bar().encode(
    x=alt.X("category:N", sort="-y"),
    y=alt.Y("revenue:Q"),
    color=alt.Color("region:N"),
    tooltip=[
        alt.Tooltip("category:N"),
        alt.Tooltip("region:N"),
        alt.Tooltip("revenue:Q", format=",.2f"),
        alt.Tooltip("order_count:Q"),
    ],
).properties(width=640, height=360)